In [1]:
from utils import MySQLAgent
import json

In [2]:
def read_config(path):
    try:
        with open(path, 'r') as file:
            configs = json.load(file)

        return configs
    except FileNotFoundError:
        print(f"The file {path} was not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file {path}.")

In [21]:
configs = read_config(path="./connections.json")

In [22]:
job_configs = configs["CREDITREPORT"]['VM1_mysql_conn_info']
sql_agent = MySQLAgent(job_configs)

In [23]:
job_configs

{'user': 'cust-update',
 'pw': 'yfy0109!',
 'host': '138.3.214.21',
 'port': 3306,
 'database': 'CrawlerDB'}

In [24]:
query = """
    select * from company
"""
df_company = sql_agent.read_table(query=query)
df_company.head()

,business_accounting_no,company_name,internal_id,query_name
0,00700019,國昌綜合工廠有限公司,K7913,None
1,00713181,華聯食品工業股份有限公司,J5984,None
2,00784930,建寶食品股份有限公司,K4395,None
3,00966299,財團法人台北市瑠公農業產銷基金會,K9232,None
4,01055919,新光醫療財團法人,K1282,None


In [25]:
i = 0
company = df_company.iloc[i]
company_account = company.business_accounting_no
company_name = company.company_name
internal_id = company.internal_id
company_account

'00700019'

In [42]:
company_name

'國昌綜合工廠有限公司'

#### companyinfo01

- 企業狀態
- 地址關聯
- 資本額(Captial_Stock_Amount)
- 最近更新日期

In [28]:
try:
    query = f"""
        select * from companyinfo01
        where Business_Accounting_No = {company_account}
    """
    companyinfo01 = sql_agent.read_table(query=query)
    companyinfo01.head()
except Exception as e:
    print("An error occurred:", e)


In [29]:
companyinfo01

,Business_Accounting_NO,Company_Status_Desc,Company_Name,Capital_Stock_Amount,Paid_In_Capital_Amount,Responsible_Name,Company_Location,Register_Organization_Desc,Company_Setup_Date,Change_Of_Approval_Data,Revoke_App_Date,Case_Status,Case_Status_Desc,Sus_App_Date,Sus_Beg_Date,Sus_End_Date,CreateTime,UpdateTime
0,00700019,核准設立,國昌綜合工廠有限公司,6000000,0,李貴發,基隆市七堵區泰安路111-1號1樓,經濟部中部辦公室,0610927,1120411,,,,,,,2024-01-29 06:22:05,2024-03-15 19:00:07


In [44]:
# 公司狀態
company_status = companyinfo01['Company_Status_Desc'].values[0]

# 地址關聯 - neo4j

# 資本額
company_captial = companyinfo01['Capital_Stock_Amount'].values[0]

In [45]:
company_captial

6000000

In [37]:
# 地址關聯 - neo4j

['核准設立']
